# Network Pair-Correlation Function

In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq
import pandas as pd
import scipy.spatial.distance as spd
import seaborn as sb

sb.set_theme()

def distanceWBoundary(p1,p2):
    x1,y1 = p1
    x2,y2 = p2
    dists = []
    dists.append(distance(x1,y1,x2,y2))
    dists.append(distance(x1,y1,x2-1,y2))
    dists.append(distance(x1,y1,x2+1,y2))
    dists.append(distance(x1,y1,x2,y2-1))
    dists.append(distance(x1,y1,x2,y2+1))
    dists.append(distance(x1,y1,x2-1,y2-1))
    dists.append(distance(x1,y1,x2-1,y2+1))
    dists.append(distance(x1,y1,x2+1,y2-1))
    dists.append(distance(x1,y1,x2+1,y2+1))
    return min(dists)

netDict = {0:'HIP', 1:'Poisson', 2:'RSA', 3:'Stealthy0.20', 4:'Stealthy0.40',5:'Stealthy0.49', 6:'Gaussian'}
netType = 'delaunay'

figNum = 0
Rs = np.concatenate([np.arange(0.0005,0.1,0.0005),np.arange(0.1,1,0.005)])
resultDF = pd.DataFrame()
resultDFEuc = pd.DataFrame()
for netNum in [0]:
    print(netDict[netNum])
    allNormalizedAvgs = []
    allNormalizedAvgsEuc = []
    if(netNum == 1):
        numTopos = 6
    else:
        numTopos = 10
    for topoNum in range(numTopos):
        if(netNum == 6 and topoNum == 4):
            continue
        print(topoNum)
        num = "%02d"%(topoNum+1)

        G = nx.read_edgelist("Networks/"+netDict[netNum]+'_'+num+'_'+netType.lower()+'_edgelist.txt', nodetype = int, data = [('distance',float)])
        if(netNum > 2):
            for E in G.edges():
                G.edges()[E]['distance'] /= 100
        allPaths = dict(nx.all_pairs_dijkstra_path_length(G, weight = 'distance'))
        allNums = []
        for n in allPaths:
            nums = list(np.histogram(list(allPaths[n].values()), bins = Rs)[0])
            allNums.append(nums)
        allNumsArr = np.array(allNums)
        avgNums = np.average(allNumsArr, axis = 0)
        normalizedAvgNums = [avgNums[i]/np.pi/((Rs[i+1])**2-Rs[i]**2)/10000 for i in range(len(avgNums))]
        allNormalizedAvgs.append(normalizedAvgNums)

        f = open("Networks/"+netDict[netNum]+'-'+num+'.txt','r')
        lines = f.readlines()
        f.close()
        if(netNum == 6):
            if(topoNum == 0):
                coords = np.array([x.rstrip().split('\t') for x in lines if x.rstrip().split(' ') != ['']]).astype(float)
            else:
                coords = [(float(x.lstrip().rstrip().split(' ')[0]),float(x.lstrip().rstrip().split(' ')[-1])) for x in lines if x.rstrip().split(' ') != ['']]
        else:
            coords = np.array([x.rstrip().split('\t') for x in lines if x.rstrip().split(' ') != ['']]).astype(float)
        
        if(netNum > 2):
            coords = coords/100
            
        #Calculate Euclidean distances between all pairs of points
        eucDistArr = spd.squareform(spd.pdist(coords,distanceWBoundary))

        allNums = []
        for i in range(len(eucDistArr)):
            nums = list(np.histogram(eucDistArr[i],bins=Rs)[0])
            allNums.append(nums)
        allNumsArr = np.array(allNums)
        avgNums = np.average(allNumsArr, axis = 0)
        normalizedAvgNums = [avgNums[i]/np.pi/((Rs[i+1])**2-Rs[i]**2)/10000 for i in range(len(avgNums))]
        allNormalizedAvgsEuc.append(normalizedAvgNums)
    
    avgNormalizedAvgs = np.average(allNormalizedAvgs, axis = 0)
    avgNormalizedAvgsEuc = np.average(allNormalizedAvgsEuc, axis=0)
    
    plt.figure(figNum, dpi = 200)
    figNum += 1
    plt.plot(Rs[:-1],avgNormalizedAvgs,label="Network")
    df = pd.DataFrame(avgNormalizedAvgs,index = Rs[:-1], columns = [netDict[netNum]])
    resultDF = pd.concat([resultDF, df],axis = 1)

    plt.plot(Rs[:-1],avgNormalizedAvgsEuc,label="Point Pattern")
    plt.xlabel('r')
    plt.ylabel('g(r)')
    plt.title(netDict[netNum])
    plt.legend(bbox_to_anchor = (1,1))
    df = pd.DataFrame(avgNormalizedAvgsEuc,index = Rs[:-1], columns = [netDict[netNum]])
    resultDFEuc = pd.concat([resultDFEuc, df],axis = 1)
    

HIP
0


NameError: name 'distance' is not defined